### sql-cheatsheets
![sql cheat sheet page1](sql-page-1.png)

![sql cheat sheet page2](sql-page-2.png)

### Review Banking DB

[schema](banks/schema.sql)

[seed](banks/seed.sql)

### Query

-- who are the customers names paid through Wells Fargo?
```sql 
CREATE VIEW payment_customer_bank AS
(SELECT p.*, c.first_name, c.last_name, b.bank_name
FROM payments p
JOIN customer c ON p.customer_id = c.customer_id
JOIN banks b ON p.bank_routing_number = b.bank_routing_number)
WHERE bank_name = 'Wells Fargo';

SELECT * from payment_customer_bank
WHERE bank_name = 'Wells Fargo'
```



[Sakila DB](https://dev.mysql.com/doc/sakila/en/sakila-structure-tables.html)

-- Identify customers who rented the film 'PANIC CLUB'.
```sql
SELECT c.customer_id, c.first_name, c.last_name, f.title
FROM customer c
JOIN rental r ON c.customer_id  = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
WHERE f.title = 'PANIC CLUB'
```